In [4]:
!git clone https://github.com/jubs12/mt-dnn_port.git -b seed
%cd mt-dnn_port

Cloning into 'mt-dnn_port'...
remote: Enumerating objects: 8652, done.
remote: Counting objects: 100% (8652/8652), done.
remote: Compressing objects: 100% (7856/7856), done.
remote: Total 63042 (delta 1234), reused 8082 (delta 743), pack-reused 54390
Receiving objects: 100% (63042/63042), 493.87 MiB | 23.16 MiB/s, done.
Resolving deltas: 100% (26338/26338), done.
Checking out files: 100% (27943/27943), done.
/content/mt-dnn_port


In [0]:
mode = 'st-dnn/assin2-sts'
pretrained = 'bert_base'

In [0]:
import os
import json
import copy
import pprint

import numpy as np
from scipy import stats

prefix = f'output/{mode}/{pretrained}'

seeds_dir = f'{prefix}/seed'
seeds = os.listdir(seeds_dir)
sample_seed = seeds[0]

grad_norm_seed_sample_dir = f'{seeds_dir}/{sample_seed}/grad_norm/'
grad_norms = os.listdir(grad_norm_seed_sample_dir)
sample_grad_norm = grad_norms[0]

dropout_grad_norm_seed_sample_dir =  \
    f'{seeds_dir}/{sample_seed}/grad_norm/{sample_grad_norm}/dropout/'
dropouts = os.listdir(dropout_grad_norm_seed_sample_dir)
sample_dropout = dropouts[0]


datasets = [f for f in os.listdir(f"{grad_norm_seed_sample_dir}/{sample_grad_norm}") 
            if f.endswith('test_scores_4.json')] 

output_dir = prefix + '/ensemble/'

cabezudo = [
            'best-pt',
            'random-pt',
            'worst-pt',
            'assin1-rte'
]

def is_rte(dataset: str) -> bool:
    rte_on_name = dataset.endswith('rte_test_scores_4.json')
    is_cabezudo = dataset in cabezudo

    return rte_on_name or is_cabezudo

def get_key(dataset: str) -> str:
    key = 'predictions' if is_rte(dataset) else 'scores'
    return key

def get_info(dataset: str, seed: str, grad_norm: str, dropout: str) -> dict:
    filename = f'{prefix}/seed/{seed}/grad_norm/{grad_norm}/dropout/{dropout}/{dataset}'

    with open(filename) as f:
        info = json.load(f)

    return info

def average_score(dataset: str, info_lst: list) -> list:
    scores_lst = [info[get_key(dataset)] for info in info_lst]

    scores_arr = np.array(scores_lst)
    scores_avg = stats.mode(scores_arr, axis=0).mode[0] if is_rte(dataset) \
                else np.mean(scores_arr, axis=0)
    scores_avg = scores_avg.tolist()
    assert len(scores_avg) == len(scores_lst[0])

    return scores_avg

if not os.path.exists(output_dir):
        os.makedirs(output_dir)

for dataset in datasets:
    info_lst = [get_info(dataset, seed, grad_norm, dropout) 
                for seed in seeds for grad_norm in grad_norms for dropout in dropouts]
    
    ensemble = copy.deepcopy(get_info(dataset, sample_seed, sample_grad_norm, sample_dropout))
    ensemble[get_key(dataset)] = average_score(dataset, info_lst)

    with open(f'{output_dir}/{dataset}', 'w') as f:
        json.dump(ensemble, f)